# Ralphy Núñez Mercado

## T2.3: Scrapear personaxes e capítulos de Rick e Morty

Scraping nivel 0. Servizos web
Apuntes relacionados: https://github.com/jfsanchez/SBD/blob/main/notebooks/scraping/scraping-0-API-REST.ipynb

Exercicio: Das URL seguintes podes obter os episodios e personaxes.

Capítulos: https://rickandmortyapi.com/api/episode
Personaxes: https://rickandmortyapi.com/api/character
Queremos saber por unha banda as personaxes e por outra os capítulos.

Coidado! A API ten un paxinador:

https://rickandmortyapi.com/api/character?page=42
https://rickandmortyapi.com/api/episode?page=3
Tes que recorrer as páxinas ata que en info, next sexa null.

O código debe ser xenérico e ser chamado cunha función tanto para personaxes como para episodios.

Deben gardarse os datos nunha base de datos de PostgreSQL.

### Imports necesarios

In [18]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json

## Creando a cadea de conexión

In [19]:
#Datos de conexión
db_host = "10.133.28.46" # ip cesga
db_port=5432 
db_user = "postgres"
db_passwd="Cl431Ns3gur4"
db_name="rickymorty"

## Recorrer la API

In [24]:
URL_episodio = "https://rickandmortyapi.com/api/episode"
URL_personaje = "https://rickandmortyapi.com/api/character" 

def recorrer(URL):
    paginas = requests.get(URL).json()["info"]["pages"] 
    resultados = []

    for i in range(1, paginas + 1):
        response = requests.get(f"{URL}?page={i}").json()
        resultados.extend(response["results"])
    
    return resultados  

episodios = recorrer(URL_episodio)
print(f"filas : {len(episodios)}")


filas : 51


(en los episodio guardar  los personajes pero en los personajes no guardar los episodios)

## Creacion de la base de datos postgresql rick y morty

### Primero crearemos el Docker   




```bash
docker run 
\ --name de-postre-sql 
\ -v /home/cesgaxuser/BBDD:/BBDDdocker 
\ -e POSTGRES_PASSWORD=Cl431Ns3gur4 
\ --restart unless-stopped 
\ -p 5432:5432 
\ -p 5433:5433 
\ -d postgres
```


### Entramos en el docker y creamos la base de datos  

```bash
docker exec -it de-postre-sql psql -U postgres
```

### Creación de las tablas para nuestra BD

Episodio:

```sql
CREATE TABLE episodio (id INT, 
    name VARCHAR(255), 
    air_date VARCHAR(255), 
    episode VARCHAR(255), 
    characters text[], 
    url text, 
    created text);
```
Personaje: 

```sql
CREATE TABLE personaje (id INT,   
    name VARCHAR(255), 
    status VARCHAR(255), 
    species VARCHAR(255), 
    type VARCHAR(255), 
    gender VARCHAR(255), 
    origin TEXT, 
    location TEXT, 
    image text, 
    url text, 
    created text);
```

### Ahora tendremos que insertar los datos de la API a las tablas de nuestra base de datos de PostgreSQL ( OJO que en la tabla personaje no vamos a guardar los episodios )

### Primero vamos a probar la conexion con la base de datos en postgreSQL

Anteriormente usabamos una conection string como esta:
```python 
    connectionString=f'mysql+pymysql://{db_user}:{db_passwd}@{db_host}:{db_port}/{db_name}'
```
El problema es que esta connectionString es para mysql y en este ejercicio usamos postgreSQL.  

Para poder conectarnos a una BD en postgreSQL tendremos que instalar esta librería: 

In [ ]:
%pip install psycopg2 
# %conda install anaconda::psycopg2

Con la libreria instalada esta sería la connectionString adecuada para conectarnos.

In [25]:
connectionString = f'postgresql+psycopg2://{db_user}:{db_passwd}@{db_host}:{db_port}/{db_name}'

In [26]:
from sqlalchemy import create_engine, text
from sqlalchemy import insert

engine = create_engine(connectionString)

selectSQL = "SELECT * FROM episodio"
dfPanda = pd.read_sql(selectSQL, engine)
print(dfPanda.head())


Empty DataFrame
Columns: [id, name, air_date, episode, characters, url, created]
Index: []


Creacion de las tablas desde python

In [66]:
create_table_episodioSQL = "CREATE TABLE episodio (id INT, name VARCHAR(255), air_date VARCHAR(255), episode VARCHAR(255), characters text[], url text, created text);"
create_table_personajeSQL = "CREATE TABLE personaje (id INT, name VARCHAR(255), status VARCHAR(255), species VARCHAR(255), type VARCHAR(255), gender VARCHAR(255), origin TEXT, location TEXT, image text, url text,created text);"

with engine.connect() as conn:
    conn.execute(text(create_table_episodioSQL))
    conn.execute(text(create_table_personajeSQL))
    conn.commit()


Ahora ya podremos insertar los datos en las tablas.

In [67]:
json_episodios = recorrer(URL_episodio)
json_personajes = recorrer(URL_personaje)

#def insert_datos(campos,valores,tabla):


with engine.connect() as conn:
    for episodio in json_episodios:
        id = episodio['id']
        name = episodio['name'].replace("'","")
        air_date = episodio['air_date']
        episode = episodio['episode']
        characters = episodio['characters']
        url = episodio['url']
        created = episodio['created']
        
        sql = f"""
            INSERT INTO episodio ( id, name, air_date, episode, characters, url, created) 
            VALUES ({id}, '{name}', '{air_date}', '{episode}', array{str(characters)}, '{url}', '{created}')
        """

        conn.execute(text(sql))
    conn.commit()


with engine.connect() as conn:
    for personaje in json_personajes:
        id = personaje['id']
        name = personaje['name'].replace("'","")
        status = personaje['status'].replace("'","")
        species = personaje['species'].replace("'","")
        typep = personaje['type'].replace("'","")
        gender = personaje['gender'].replace("'","")
        origin = personaje['origin']['name'].replace("'","")
        location = personaje['location']['name'].replace("'","")
        image = personaje['image']
        url = personaje['url']
        created = personaje['created']

         
        sql = f"""
            INSERT INTO personaje ( id, name, status, species, type, gender, origin, location, image, url,created) 
            VALUES ({id}, '{name}', '{status}', '{species}', {f"'{typep}'" if typep != 'NULL' else 'NULL'}, '{gender}','{location}','{origin}' ,'{image}','{url}','{created}')
        """
        conn.execute(text(sql))
    conn.commit()

### Comprobamos que los datos se han insertado correctamente

In [17]:
count = "SELECT count(*) FROM episodio "
dfCount = pd.read_sql(count, engine)
print(dfCount)
dfPanda = pd.read_sql(selectSQL,engine)
dfPanda

   count
0     51


,id,name,air_date,episode,characters,url,created
0,1,Pilot,"December 2, 2013",S01E01,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/1,2017-11-10T12:56:33.798Z
1,2,Lawnmower Dog,"December 9, 2013",S01E02,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/2,2017-11-10T12:56:33.916Z
2,3,Anatomy Park,"December 16, 2013",S01E03,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/3,2017-11-10T12:56:34.022Z
3,4,M. Night Shaym-Aliens!,"January 13, 2014",S01E04,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/4,2017-11-10T12:56:34.129Z
4,5,Meeseeks and Destroy,"January 20, 2014",S01E05,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/5,2017-11-10T12:56:34.236Z
5,6,Rick Potion #9,"January 27, 2014",S01E06,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/6,2017-11-10T12:56:34.339Z
6,7,Raising Gazorpazorp,"March 10, 2014",S01E07,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/7,2017-11-10T12:56:34.441Z
7,8,Rixty Minutes,"March 17, 2014",S01E08,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/8,2017-11-10T12:56:34.543Z
8,9,Something Ricked This Way Comes,"March 24, 2014",S01E09,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/9,2017-11-10T12:56:34.645Z
9,10,Close Rick-counters of the Rick Kind,"April 7, 2014",S01E10,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/10,2017-11-10T12:56:34.747Z


In [68]:
count = "SELECT count(*) FROM personaje "
dfCount = pd.read_sql(count, engine)
print(dfCount)
select_personaje = "SELECT * FROM personaje "
dfselect_personaje = pd.read_sql(select_personaje,engine)
dfselect_personaje

   count
0    826


,id,name,status,species,type,gender,origin,location,image,url,created
0,1,Rick Sanchez,Alive,Human,,Male,Citadel of Ricks,Earth (C-137),https://rickandmortyapi.com/api/character/avat...,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z
1,2,Morty Smith,Alive,Human,,Male,Citadel of Ricks,unknown,https://rickandmortyapi.com/api/character/avat...,https://rickandmortyapi.com/api/character/2,2017-11-04T18:50:21.651Z
2,3,Summer Smith,Alive,Human,,Female,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,https://rickandmortyapi.com/api/character/3,2017-11-04T19:09:56.428Z
3,4,Beth Smith,Alive,Human,,Female,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,https://rickandmortyapi.com/api/character/4,2017-11-04T19:22:43.665Z
4,5,Jerry Smith,Alive,Human,,Male,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,https://rickandmortyapi.com/api/character/5,2017-11-04T19:26:56.301Z
...,...,...,...,...,...,...,...,...,...,...,...
821,822,Young Jerry,unknown,Human,,Male,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,https://rickandmortyapi.com/api/character/822,2021-11-02T17:18:31.934Z
822,823,Young Beth,unknown,Human,,Female,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,https://rickandmortyapi.com/api/character/823,2021-11-02T17:19:00.951Z
823,824,Young Beth,unknown,Human,,Female,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,https://rickandmortyapi.com/api/character/824,2021-11-02T17:19:47.957Z
824,825,Young Jerry,unknown,Human,,Male,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,https://rickandmortyapi.com/api/character/825,2021-11-02T17:20:14.305Z


## MONGO DB

Para conectarnos a mongoDB, una vez más deberemos descargar la libreria para poder utilizar su respectiva conectionString

In [ ]:
!conda install -y pymongo dnspython

^C


Retrieving notices: done
Channels:
 - defaults
 - conda-forge
 - anaconda
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\ralphy.nunezmercado\AppData\Local\miniconda3\envs\bigdata

  added / updated specs:
    - dnspython
    - pymongo


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    dnspython-2.4.2            |  py312haa95532_0         480 KB
    pymongo-4.8.0              |  py312h827c3e9_0         1.3 MB
    ------------------------------------------------------------
                                           Total:         1.7 MB

The following NEW packages will be INSTALLED:

  dnspython          pkgs/main/win-64::dnspython-2.4.2-py312haa95532_0 
  pymongo            pkgs/main/win-64::pymongo-4.8.0-py312h827c3e9_0 



pymongo-4.8.0        | 1.3 MB    |            |   0% 

dnspython-2.4.2      | 480 KB    |            |

In [ ]:
from pymongo import MongoClient

#HOST="(...).mongodb.net"
HOST="localhost"
PORT=27017
USERNAME="ralphynm1000"
PASSWORD="hola1234"

if HOST == 'localhost':
    if not USERNAME:
        cli_mongo = MongoClient(HOST, PORT)
    else:
        cli_mongo = MongoClient(HOST, PORT, USERNAME, PASSWORD)
else:
    cli_mongo = MongoClient(f"mongodb+srv://{USERNAME}:{PASSWORD}@{HOST}/")

# mongodb+srv://ralphynm1000:<db_password>@cluster0.6ej71.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0